# Strands Agent SDK

## 0. Jupyter 환경 설정

> **🎯 목적**: 개발 효율성을 위한 자동 리로드 기능을 활성화합니다.

| 매직 명령어 | 기능 | 효과 |
|------------|------|------|
| `%load_ext autoreload` | autoreload 확장 로드 | 자동 리로드 기능 활성화 |
| `%autoreload 2` | 전체 모듈 자동 리로드 | 모든 모듈 변경사항 실시간 반영 |

💡 **Tip**: 이 설정으로 코드를 수정할 때마다 커널을 재시작하지 않아도 변경사항이 바로 적용됩니다!

In [1]:
%load_ext autoreload
%autoreload 2

## 1. 환경변수 및 lib path 설정

### 1.1 필요한 라이브러리 import

> **🎯 목적**: 프로젝트 환경 설정을 위한 핵심 라이브러리들을 준비합니다.

| 라이브러리 | 역할 | 사용 목적 |
|-----------|------|-----------|
| `os` | 운영체제 인터페이스 | 환경변수 접근, 시스템 정보 |
| `sys` | Python 인터프리터 제어 | 모듈 경로 설정 |
| `dotenv` | 환경변수 관리 | `.env` 파일에서 설정 로드 |

💡 **Tip**: `.env` 파일을 사용하면 AWS 인증 정보 등 민감한 설정을 안전하게 관리할 수 있습니다.

In [2]:
import os, sys
from dotenv import load_dotenv

### 1.2 환경 변수 로드 및 모듈 경로 설정

> **🎯 목적**: AWS 연결과 프로젝트 모듈 접근을 위한 환경을 구성합니다.

| 단계 | 작업 | 설명 |
|------|------|------|
| 1️⃣ | `.env` 파일 로드 | 환경변수를 안전하게 불러옴 |
| 2️⃣ | AWS 리전 확인 | Bedrock 서비스 연결 지역 검증 |
| 3️⃣ | 모듈 경로 추가 | 상위 디렉토리의 `src` 모듈 접근 허용 |

💡 **Tip**: `AWS_DEFAULT_REGION`이 출력되면 환경 설정이 올바르게 완료된 것입니다!

In [3]:
load_dotenv()
print (f'AWS_DEFAULT_REGION: {os.getenv("AWS_DEFAULT_REGION")}')

module_path = ".."
sys.path.append(os.path.abspath(module_path))

AWS_DEFAULT_REGION: us-west-2


## 2. Utilities

### 2.1 Get llm model
- function name: "get_model"
- script path: ["../src/utils/strands_sdk_utils.py"](../src/utils/strands_sdk_utils.py#L56-L100)

### 2.2 Get system prompt
- function name: "apply_prompt_template"
- script path: ["../src/prompts/template.py"](../src/prompts/template.py#L4-L12)

### 2.3 Create agent
- function name: "get_agent"
- script path: ["../src/utils/strands_sdk_utils.py"](../src/utils/strands_sdk_utils.py#L102-L126)

## 3. Agent definition

### 3.1 에이전트 이름 정의

> **🎯 목적**: 튜토리얼 에이전트의 고유 식별자를 설정합니다.

| 속성 | 값 | 용도 |
|------|-----|------|
| **에이전트명** | `toy_agent` | 시스템 프롬프트 파일명 참조 |

💡 **Tip**: 에이전트 이름은 프로젝트 전반에서 일관되게 사용되는 중요한 식별자입니다!

In [4]:
agent_name = "toy_agent"

### 3.2 시스템 프롬프트 (System prompt) 파일 생성

> **🎯 목적**: `%%writefile` 매직 명령어로 에이전트의 핵심 지침을 파일로 저장합니다.

| 구성 요소 | 내용 | 목적 |
|----------|------|------|
| **🤖 에이전트 정체성** | Bedrock-Manus 역할 정의 | 친근한 AI 어시스턴트 성격 설정 |
| **🛠️ 도구 설명** | Python REPL, Bash 도구 | 각 도구의 사용 시점과 방법 안내 |
| **📋 사용 가이드라인** | 도구 선택 기준 | 상황별 적절한 도구 선택 로직 |
| **💬 응답 스타일** | 대화 방식 정의 | 사용자 친화적 커뮤니케이션 규칙 |

💡 **Tip**: `%%writefile`로 생성된 프롬프트는 에이전트의 "두뇌" 역할을 하는 중요한 설정 파일입니다!

In [5]:
%%writefile ../src/prompts/toy_agent.md

---
CURRENT_TIME: {CURRENT_TIME}
AGENT_NAME: {AGENT_NAME}
---

You are Bedrock-Manus, a friendly AI assistant developed by AWS AIML Specialist SA Dongjin Jang.
You specialize in handling greetings, small talk, and knowledge-based question answering using available tools.

## Available Tools

You have access to the following tools that you should use when appropriate:

### 1. Python REPL Tool (python_repl_tool)
**When to use**: Use this tool when users need to execute Python code or perform data analysis:
- Running Python scripts or code snippets
- Data analysis and calculations
- Testing code functionality
- Mathematical computations

**What it does**: Executes Python code in a REPL environment and returns the output

**Input**: Python code string

### 2. Bash Tool (bash_tool) 
**When to use**: Use this tool when users need to execute system commands or perform file operations:
- Running shell commands
- File system operations (ls, mkdir, etc.)
- System information queries
- Development tasks requiring command line operations

**What it does**: Executes bash commands and returns the output

**Input**: A bash command string

## Tool Usage Guidelines

1. **Assess the user's request** - Determine if the question requires tool usage
2. **Choose the appropriate tool** - Select based on the type of information needed
3. **Use RAG tool for knowledge queries** - When the user asks about topics that might be in your knowledge base
4. **Use Python REPL for code execution** - When the user needs to run Python code or perform calculations
5. **Use Bash tool for system operations** - When the user needs to interact with the system
6. **Provide helpful responses** - Always explain the results in a user-friendly way

## Response Style

- Be friendly and conversational
- Provide clear, helpful answers
- When using tools, explain what you're doing and why
- If a tool doesn't provide the needed information, acknowledge this and offer alternatives
- Always prioritize user experience and clarity

Remember to use tools proactively when they can help answer user questions more accurately or completely.

Overwriting ../src/prompts/toy_agent.md


### 3.3 에이전트 생성

> **🎯 목적**: Strands SDK를 사용하여 설정이 완료된 실제 에이전트를 생성합니다.

| 파라미터 | 설정값 | 설명 |
|----------|--------|------|
| **🤖 agent_type** | `claude-sonnet-3-7` | 사용할 LLM 모델 (고성능 추론) |
| **🧠 enable_reasoning** | `False` | 추론 기능 비활성화 (빠른 응답) |
| **⚡ prompt_cache_info** | `(False, None)` | 프롬프트 캐싱 비활성화 |
| **📡 streaming** | `True` | 실시간 응답 스트리밍 활성화 |

💡 **Tip**: 이 설정으로 기본적인 대화형 에이전트가 생성됩니다. 도구는 아직 포함되지 않았습니다!

In [10]:
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template

In [11]:
agent = strands_utils.get_agent(
    agent_name=agent_name,
    system_prompts=apply_prompt_template(prompt_name=agent_name, prompt_context={"AGENT_NAME": agent_name}),
    agent_type="claude-sonnet-3-7", # claude-sonnet-3-5-v-2, claude-sonnet-3-7
    enable_reasoning=False,
    prompt_cache_info=(False, None), #(False, None), (True, "default")
    streaming=True,
)

#system_prompts=apply_prompt_template(prompt_name=agent_name, prompt_context={"AGENT_NAME": agent_name})
#print (f'System prompt: \n{system_prompts}')

INFO:src.utils.strands_sdk_utils:TOY_AGENT - Prompt Cache Disabled


## 4. Invocation

### 4.1 기본 에이전트 호출 (without streaming)

> **🎯 목적**: 생성된 에이전트와의 첫 번째 대화를 테스트하며 응답 수집 방식을 학습합니다.

| 처리 단계 | 방법 | 설명 |
|----------|------|------|
| **📨 메시지 전송** | `process_streaming_response_yield()` | 비동기 스트리밍 응답 처리 |
| **🔄 응답 수집** | `event.get("text_chunk")` | 스트리밍 텍스트 조각들을 수집 |
| **📋 결과 출력** | `full_text` 변수 | 완성된 전체 응답만 최종 출력 |

🔗 **참조**: [`process_streaming_response_yield`](../src/utils/strands_sdk_utils.py#L187-L239)

#### 📤 스트리밍 변환 결과 예시
**"안녕하세요" 응답의 스트리밍 과정**

```python
# 스트림 1:
{"data": "안녕"} → {
    "timestamp": "2024-01-15T10:30:00.123456",
    "session_id": "session-1", 
    "agent_name": "toy_agent",
    "type": "agent_text_stream",
    "event_type": "text_chunk", 
    "data": "안녕"
}

# 스트림 2:
{"data": "하세요"} → {
    "timestamp": "2024-01-15T10:30:00.145678",
    "session_id": "session-1",
    "agent_name": "toy_agent", 
    "type": "agent_text_stream",
    "event_type": "text_chunk",
    "data": "하세요"
}
```

**핵심**: 각 토큰이 개별 이벤트로 변환되어 실시간 스트리밍이 가능하며, 타임스탬프로 순서를 보장합니다.

💡 **Tip**: 스트리밍을 받지만 최종 결과만 보는 방식으로, 완성된 응답을 한 번에 확인할 수 있습니다!

In [8]:
user_input = "안녕 만나서 반가워. 나는 지금 Strands Agents SDK 공부중이야."

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent,
    message=user_input,
    agent_name=agent_name,
    source=agent_name
):  
    #print (event)
    if event.get("event_type") == "text_chunk":
        full_text += event.get("data", "")

response = {"text": full_text}

print (f'\nResponse: {response['text']}')


Response: 안녕하세요! 만나서 반갑습니다. Strands Agents SDK를 공부하고 계시는군요, 정말 흥미로운 주제입니다!

Strands Agents SDK는 AWS Bedrock에서 제공하는 에이전트 개발 도구로, 대규모 언어 모델(LLM)을 활용하여 다양한 도구와 상호작용할 수 있는 AI 에이전트를 만드는 데 도움을 줍니다.

공부하시면서 특별히 알고 싶은 부분이나 도움이 필요한 부분이 있으신가요? Python 코드 예제를 실행해보거나, 특정 기능에 대해 더 자세히 알아보고 싶으시다면 말씀해 주세요. 도움을 드릴 수 있어 기쁩니다!


### 4.2 실시간 스트리밍 응답 테스트

> **🎯 목적**: 실시간으로 응답이 생성되는 과정을 시각적으로 확인하며 스트리밍의 장점을 체험합니다.

| 처리 방식 | 함수 | 효과 |
|----------|------|------|
| **🎬 실시간 출력** | `process_event_for_display()` | 텍스트가 타이핑되는 것처럼 실시간 표시 |
| **📊 이벤트 분석** | 이벤트 타입별 처리 | 텍스트, 도구 사용 등 다양한 이벤트 구분 |
| **💾 응답 수집** | `full_text` 누적 | 최종 완성된 응답도 별도 저장 |

🔗 **참조**: [`process_event_for_display`](../src/utils/strands_sdk_utils.py#L333-L379)

💡 **Tip**: 텍스트가 실시간으로 나타나는 효과를 구현할 수 있습니다!

In [9]:
message = "Strands Agents SDK의 장점은 뭐야?"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    strands_utils.process_event_for_display(event)

    if event.get("event_type") == "text_chunk":
        full_text += event.get("data", "")
    
response = {"text": full_text}

print (f'\nResponse: {response['text']}')

Strands Agents SDK의 주요 장점은 다음과 같습니다:

1. **간편한 에이전트 개발**: 복잡한 AI 에이전트를 쉽게 구축할 수 있는 프레임워크를 제공합니다. 개발자는 기본 LLM 통합 작업보다 비즈니스 로직에 집중할 수 있습니다.

2. **도구 통합 용이성**: 다양한 외부 도구, API, 데이터베이스 등을 에이전트에 쉽게 연결할 수 있습니다. 이를 통해 에이전트가 실제 작업을 수행할 수 있는 능력이 크게 향상됩니다.

3. **AWS 서비스와의 원활한 통합**: AWS 생태계 내의 다른 서비스들과 자연스럽게 연동됩니다.

4. **확장성**: 간단한 챗봇부터 복잡한 업무 자동화 시스템까지 다양한 규모의 애플리케이션을 구축할 수 있습니다.

5. **유연한 커스터마이징**: 에이전트의 행동, 응답 방식, 도구 사용 방법 등을 세밀하게 조정할 수 있습니다.

6. **개발 시간 단축**: 많은 기본 기능이 이미 구현되어 있어 개발 시간을 크게 단축할 수 있습니다.

7. **강력한 LLM 기반**: AWS Bedrock의 다양한 고성능 언어 모델을 활용할 수 있습니다.

8. **메모리 및 상태 관리**: 대화 컨텍스트와 상태를 효과적으로 관리할 수 있는 기능을 제공합니다.

Strands Agents SDK를 공부하시면서 특정 기능이나 구현 방법에 대해 더 알고 싶으신 부분이 있으신가요?
Response: Strands Agents SDK의 주요 장점은 다음과 같습니다:

1. **간편한 에이전트 개발**: 복잡한 AI 에이전트를 쉽게 구축할 수 있는 프레임워크를 제공합니다. 개발자는 기본 LLM 통합 작업보다 비즈니스 로직에 집중할 수 있습니다.

2. **도구 통합 용이성**: 다양한 외부 도구, API, 데이터베이스 등을 에이전트에 쉽게 연결할 수 있습니다. 이를 통해 에이전트가 실제 작업을 수행할 수 있는 능력이 크게 향상됩니다.

3. **AWS 서비스와의 원활한 통합**: AWS 생태계 내의 다른 서비스들과 자연스럽게 연동됩니다.


## 5. Tools

### 5.1 도구(Tools) 사용을 위한 import

> **🎯 목적**: 에이전트가 실제 작업을 수행할 수 있도록 실행 도구들을 준비합니다.

| 도구 | 기능 | 사용 사례 |
|------|------|-----------|
| **🐍 python_repl_tool** | Python 코드 실행 | 데이터 분석, 계산, 시각화 |
| **⚡ bash_tool** | 시스템 명령어 실행 | 파일 조작, 시스템 정보 조회 |

💡 **Tip**: 도구를 추가하면 에이전트가 단순한 챗봇에서 실제 작업을 수행하는 AI 어시스턴트로 진화합니다!

In [ ]:
from src.tools import python_repl_tool, bash_tool

In [ ]:
agent = strands_utils.get_agent(
    agent_name=agent_name,
    system_prompts=apply_prompt_template(prompt_name=agent_name, prompt_context={"AGENT_NAME": agent_name}),
    agent_type="claude-sonnet-3-7", # claude-sonnet-3-5-v-2, claude-sonnet-3-7
    enable_reasoning=False,
    prompt_cache_info=(False, None), #(False, None), (True, "default")
    streaming=True,
    tools=[python_repl_tool, bash_tool]
)

### 5.2 Bash 도구 사용 테스트

> **🎯 목적**: 파일 시스템 탐색을 통해 bash_tool의 동작을 확인합니다.

| 작업 유형 | 사용 도구 | 실행 명령어 |
|----------|----------|-------------|
| **파일/디렉토리 조회** | `bash_tool` | `ls`, `find`, `pwd` |
| **시스템 정보 확인** | `bash_tool` | `df`, `ps`, `whoami` |
| **파일 조작** | `bash_tool` | `cp`, `mv`, `rm`, `mkdir` |

#### 📋 테스트 내용
- **작업**: `../src/prompts` 디렉토리 내용 확인
- **도구 선택 조건**: 파일 시스템 탐색 요청 시 자동으로 bash_tool 선택됨
- **예상 명령어**: `ls ../src/prompts`

💡 **Tip**: 에이전트가 "디렉토리 조회" 요청을 bash 명령어로 자동 변환합니다!

In [ ]:
message = "../src/prompts 디렉토리 조회해 주세요"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    #print (event)
    strands_utils.process_event_for_display(event)

### 5.3 Python 도구 사용 테스트

> **🎯 목적**: Python 코드 실행을 통해 python_repl_tool의 동작을 확인합니다.

| 작업 유형 | 사용 도구 | 실행 예시 |
|----------|----------|-----------|
| **계산 및 연산** | `python_repl_tool` | `2 + 2`, `math.sqrt(16)` |
| **데이터 처리** | `python_repl_tool` | `pandas.read_csv()`, `numpy.array()` |
| **코드 실행** | `python_repl_tool` | `print()`, `for loop`, 함수 정의 |

#### 📋 테스트 내용
- **작업**: "Hello world" 출력하는 Python 코드 작성 및 실행
- **도구 선택 조건**: Python 코드 실행 요청 시 자동으로 python_repl_tool 선택됨
- **예상 코드**: `print("Hello world")`

💡 **Tip**: 에이전트가 자연어 요청을 Python 코드로 변환하고 직접 실행합니다!

In [ ]:
message = "Hello world 를 프린팅하는 파이썬 코드를 작성하고 실행시켜 줄래?"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    strands_utils.process_event_for_display(event)

## 6. Built-in utility

In [ ]:
from pprint import pprint

### 6.1 Check agent

#### 6.1.1 에이전트 시스템 프롬프트 확인

> **🎯 목적**: 생성된 에이전트의 시스템 프롬프트를 확인합니다.

| 확인 항목 | 내용 |
|----------|------|
| **에이전트 지침과 역할** | 에이전트에 설정된 역할과 행동 지침 검토 |
| **템플릿 적용 여부** | 템플릿이 올바르게 적용되었는지 확인 |

💡 **Tip**: 시스템 프롬프트는 에이전트의 행동을 결정하는 핵심 설정입니다!

**System prompt**

In [ ]:
system_prompt = agent.system_prompt
pprint(system_prompt)

#### 6.1.2 에이전트 대화 기록 확인

> **🎯 목적**: 에이전트의 메시지 히스토리를 확인합니다.

| 포함 내용 | 설명 |
|----------|------|
| **모든 대화 내용** | 지금까지의 대화 내용과 도구 사용 기록 |
| **다양한 메시지 타입** | 사용자 메시지, 에이전트 응답, 도구 호출 결과 등 |

💡 **Tip**: 메시지 히스토리를 통해 에이전트가 어떤 과정을 거쳐 응답했는지 추적할 수 있습니다!

**Message history**

In [ ]:
agent_messages = agent.messages
pprint(agent_messages)

#### 6.1.3 에이전트 성능 메트릭 확인

> **🎯 목적**: 에이전트의 이벤트 루프 메트릭을 확인합니다.

| 메트릭 유형 | 내용 |
|------------|------|
| **성능 데이터** | 응답 시간, 토큰 사용량, 에러 발생 등의 성능 데이터 |
| **모니터링 활용** | 에이전트 운영 상태와 효율성을 모니터링하는 데 유용 |

💡 **Tip**: 성능 메트릭을 통해 에이전트의 효율성과 안정성을 분석할 수 있습니다!

**Observability**

In [ ]:
pprint(agent.event_loop_metrics)

#### 6.1.4 에이전트 상태 복원을 위한 라이브러리 import

> **🎯 목적**: 기존 에이전트의 상태를 사용하여 새로운 에이전트를 생성하기 위한 라이브러리들을 import합니다.

| 라이브러리 | 용도 |
|----------|------|
| **Strands 기본 클래스들** | Agent, BedrockModel 등 핵심 컴포넌트 |
| **Bedrock 모델과 설정 정보** | 시스템 프롬프트, 메시지 히스토리 복원 |

💡 **Tip**: 에이전트 상태 복원을 통해 대화의 연속성을 유지할 수 있습니다!

**Resume**

In [ ]:
from strands import Agent
from botocore.config import Config
from strands.models import BedrockModel
from src.utils.bedrock import bedrock_info

In [ ]:
llm_ = BedrockModel(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-7-Sonnet-CRI"),
    streaming=True,
    max_tokens=8192,
    stop_sequencesb=["\n\nHuman"],
    temperature=0.01,
    cache_prompt=None, # None/ephemeral/defalut
    #cache_tools: Cache point type for tools
    boto_client_config=Config(
        read_timeout=900,
        connect_timeout=900,
        retries=dict(max_attempts=50, mode="standard"),
    )
)


agent_ = Agent(
    model=llm_,
    tools=[python_repl_tool, bash_tool],
    system_prompt=system_prompt,
    messages=agent_messages,
    callback_handler=None # async iterator로 대체 하기 때문에 None 설정
)

In [ ]:
message = "이어서 대화 하는거 맞니?"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent_,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    strands_utils.process_event_for_display(event)


### 6.2 [Conversation management](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/?h=conversa)

As conversations grow, managing this context becomes increasingly important for several reasons:

- **Token Limits**: Language models have fixed context windows (maximum tokens they can process)
- **Performance**: Larger contexts require more processing time and resources
- **Relevance**: Older messages may become less relevant to the current conversation
- **Coherence**: Maintaining logical flow and preserving important information

#### 6.2.1 SlidingWindowConversationManager

슬라이딩 윈도우 방식의 대화 관리자를 구현합니다:
- 고정된 수의 최근 메시지를 유지하는 슬라이딩 윈도우 전략을 구현합니다.
- Agent 클래스에서 기본적으로 사용하는 대화 매니저입니다.
- Strands SDK의 대화 관리 기능을 사용하기 위해 `SlidingWindowConversationManager`를 import합니다.
- 이는 메모리 효율성을 위해 최근 N개의 메시지만 유지하는 방식입니다.

In [ ]:
from strands.agent.conversation_manager import SlidingWindowConversationManager

슬라이딩 윈도우 방식의 대화 관리자를 생성합니다.
- `window_size=3`: 최근 3개의 메시지만 유지
- `should_truncate_results=True`: 큰 도구 결과를 자동으로 축약
- 메모리 사용량을 제한하면서도 최근 맥락을 유지

In [ ]:
# Create a conversation manager with custom window size
conversation_manager = SlidingWindowConversationManager(
    window_size=3,  # Maximum number of messages to keep
    should_truncate_results=True, # Enable truncating the tool result when a message is too large for the model's context window 
)

### 에이전트에 대화 관리자 적용

생성된 대화 관리자를 에이전트에 설정합니다.
이제 에이전트는 설정된 윈도우 크기에 따라 대화 기록을 자동으로 관리합니다.

In [ ]:
agent.conversation_manager = conversation_manager

### 슬라이딩 윈도우 동작 테스트

새로운 대화를 시작하여 슬라이딩 윈도우가 어떻게 동작하는지 확인합니다.
- 새로운 메시지 추가 후 메시지 히스토리를 확인
- 윈도우 크기 제한으로 오래된 메시지가 제거되는지 검증

In [ ]:
message = "안녕 나는 장동진이야"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    strands_utils.process_event_for_display(event)

print ("\n")
pprint (agent.messages)

#### 6.2.2 SummarizingConversationManager

오래된 메시지를 요약하여 중요한 정보를 보존하면서 컨텍스트 한계 내에서 대화를 관리합니다.

**주요 설정:**

| 파라미터 | 타입 | 기본값 | 설명 |
|---------|------|--------|------|
| `summary_ratio` | `float` | `0.3` | 컨텍스트 축소 시 요약할 메시지 비율 (0.1~0.8 범위) |
| `preserve_recent_messages` | `int` | `10` | 항상 유지할 최근 메시지 수 |
| `summarization_agent` | `Agent` | `None` | 요약 생성용 커스텀 에이전트 (system_prompt와 동시 사용 불가) |
| `summarization_system_prompt` | `str` | `None` | 요약용 커스텀 시스템 프롬프트 (agent와 동시 사용 불가) |

> **기본 요약 방식**: 커스텀 설정이 없을 경우, 주요 토픽, 사용된 도구, 기술적 정보를 3인칭 형태의 구조화된 불릿 포인트로 요약합니다.

In [ ]:
from strands.agent.conversation_manager import SummarizingConversationManager

### SummarizingConversationManager 설정

요약 기반 대화 관리자를 생성합니다.
- `summary_ratio=0.3`: 컨텍스트 축소 시 30%의 메시지를 요약
- `preserve_recent_messages=3`: 최근 3개 메시지는 항상 유지
- `summarization_system_prompt`: 기술적 대화에 특화된 커스텀 요약 프롬프트
- 코드 변경사항, 아키텍처 결정, 기술적 솔루션에 중점을 둔 요약

In [ ]:
# Custom system prompt for technical conversations
custom_system_prompt = """
You are summarizing a technical conversation. Create a concise bullet-point summary that:
- Focuses on code changes, architectural decisions, and technical solutions
- Preserves specific function names, file paths, and configuration details
- Omits conversational elements and focuses on actionable information
- Uses technical terminology appropriate for software development

Format as bullet points without conversational language.
"""

conversation_manager = SummarizingConversationManager(
    summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
    preserve_recent_messages=3,  # Always keep 10 most recent messages
    summarization_system_prompt=custom_system_prompt
)

### 요약 관리자 적용

새로 생성한 요약 기반 대화 관리자를 에이전트에 설정합니다.
이제 에이전트는 오래된 대화를 요약하여 중요한 정보를 보존합니다.

In [ ]:
agent.conversation_manager = conversation_manager

### 요약 기능 테스트

요약 기반 대화 관리가 어떻게 동작하는지 테스트합니다.
- 새로운 대화 시작 후 메시지 히스토리 확인
- 요약 기능이 활성화되면 이전 대화들이 어떻게 요약되는지 확인

In [ ]:
message = "안녕 나는 장동진이야"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    #print (event)
    strands_utils.process_event_for_display(event)

print ("\n")
pprint (agent.messages)